In [ ]:
import os
import numpy as np
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

for i in range(1, 9):

    subj = "subj0{}".format(i)
    path = "../../data.nosync/{}".format(subj)+"/{}_split/{}_features/vit-gpt2-image-captioning/decoder-raw"
    
    print("==========================<{}>==========================".format(subj))

    for layer in [-1, -2, -3, -4]:

        print("last layer: #{}".format(-layer))

        # get all data
        feats = list()
        files = os.listdir(path.format("training", "training"))
        for x in tqdm(files):
            dat = np.load(os.path.join(path.format("training", "training"), x))
            feats.append(dat[layer].reshape(-1))

        test_files = os.listdir(path.format("test", "test"))
        for x in tqdm(test_files):
            dat = np.load(os.path.join(path.format("test", "test"), x))
            feats.append(dat[layer].reshape(-1))

        files += test_files
        feats = np.vstack(feats)
        feats = StandardScaler().fit_transform(feats)

        print("Start PCA Reduction")
        pca = PCA(n_components=512)
        pca.fit(X=feats)

        reduced_feats = pca.transform(feats)

        train_save = "../../data.nosync/{}/{}_split/{}_features/vit-gpt2-image-captioning/decoder-last{}-pca-512".format(
            subj, "training", "training", -layer)
        
        test_save = "../../data.nosync/{}/{}_split/{}_features/vit-gpt2-image-captioning/decoder-last{}-pca-512".format(
            subj, "test", "test", -layer)

        if not os.path.isdir(train_save):
            os.makedirs(train_save)

        if not os.path.isdir(test_save):
            os.makedirs(test_save)

        print("Saving...")

        for f, x in zip(reduced_feats, files):
            
            p = train_save if x.split("-")[0] == "train" else test_save
            np.save(os.path.join(p, x), f.astype(np.float32))
